In [1]:
import torch, detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

<Logger detectron2 (DEBUG)>

In [2]:
import numpy as np
import os, json, cv2, random

In [3]:
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

In [4]:
from detectron2.data.datasets import register_coco_instances
register_coco_instances("sps_dataset_train", {}, 
                        "/home/ecervera/Desktop/CERN/detectron2/jupyter-demo/annotations_contours/instances_Subset01.json", 
                        "/home/ecervera/Desktop/CERN/detectron2/jupyter-demo/image_SPS_1720795090")

In [5]:
sps_metadata = MetadataCatalog.get("sps_dataset_train")
sps_dataset = DatasetCatalog.get("sps_dataset_train")

[07/24 16:05:07 d2.data.datasets.coco]: Loaded 302 images in COCO format from /home/ecervera/Desktop/CERN/detectron2/jupyter-demo/annotations_contours/instances_Subset01.json


In [6]:
from detectron2.engine import DefaultTrainer
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("sps_dataset_train",)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 2  # This is the real "batch size" commonl
cfg.SOLVER.BASE_LR = 0.00025  # pick a good LR
cfg.SOLVER.MAX_ITER = 300	# 300 iterations seems good enough 
cfg.SOLVER.STEPS = []    	# do not decay learning rate
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128   # The "RoIHead batch size". 128 is faster, and good enough for this toy dataset (default: 512)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 2
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

[07/24 16:05:08 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

Skip loading parameter 'roi_heads.box_predictor.cls_score.weight' to the model due to incompatible shapes: (81, 1024) in the checkpoint but (3, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.cls_score.bias' to the model due to incompatible shapes: (81,) in the checkpoint but (3,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.weight' to the model due to incompatible shapes: (320, 1024) in the checkpoint but (8, 1024) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.box_predictor.bbox_pred.bias' to the model due to incompatible shapes: (320,) in the checkpoint but (8,) in the model! You might want to double check if this is expected.
Skip loading parameter 'roi_heads.mask_head.predictor.weight' to the model due to incompatible shapes: (80, 256, 1, 1) in the checkpoint but (2, 256, 1, 1) in

[07/24 16:05:08 d2.engine.train_loop]: Starting training from iteration 0


/home/ecervera/.local/lib/python3.10/site-packages/torch/functional.py:507: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3549.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


[07/24 16:05:13 d2.utils.events]:  eta: 0:00:56  iter: 19  total_loss: 2.186  loss_cls: 0.9881  loss_box_reg: 0.4243  loss_mask: 0.6923  loss_rpn_cls: 0.04807  loss_rpn_loc: 0.009546    time: 0.1988  last_time: 0.1823  data_time: 0.0064  last_data_time: 0.0023   lr: 1.6068e-05  max_mem: 2247M
[07/24 16:05:17 d2.utils.events]:  eta: 0:00:51  iter: 39  total_loss: 1.986  loss_cls: 0.802  loss_box_reg: 0.4696  loss_mask: 0.6849  loss_rpn_cls: 0.04342  loss_rpn_loc: 0.008582    time: 0.1958  last_time: 0.2040  data_time: 0.0023  last_data_time: 0.0025   lr: 3.2718e-05  max_mem: 2247M
[07/24 16:05:21 d2.utils.events]:  eta: 0:00:47  iter: 59  total_loss: 1.914  loss_cls: 0.6251  loss_box_reg: 0.5611  loss_mask: 0.6682  loss_rpn_cls: 0.03151  loss_rpn_loc: 0.009163    time: 0.1942  last_time: 0.1876  data_time: 0.0024  last_data_time: 0.0025   lr: 4.9367e-05  max_mem: 2247M
[07/24 16:05:25 d2.utils.events]:  eta: 0:00:43  iter: 79  total_loss: 1.917  loss_cls: 0.5492  loss_box_reg: 0.6784  l

In [7]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7   # set a custom testing threshold
predictor = DefaultPredictor(cfg)

[07/24 16:06:11 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from ./output/model_final.pth ...


In [8]:
from detectron2.utils.visualizer import ColorMode

In [9]:
cv2.startWindowThread()
cv2.namedWindow("SPS video")

In [10]:
input_video = cv2.VideoCapture('SPSVideo2.mp4')

In the terminal before launching jupyter-lab, run:
```
source /opt/ros/humble/setup.bash
```

In [11]:
import rclpy
from rclpy.node import Node
from sensor_msgs.msg import Image
from cv_bridge import CvBridge
import cv2

In [12]:
rclpy.init(args=None)
node = Node('publisher')
publisher = node.create_publisher(Image, 'image', 10)
br = CvBridge()

In [ ]:
try:
    while input_video.isOpened():
        ret, im = input_video.read()
        if not ret:
            break
        outputs = predictor(im)
        v = Visualizer(im[:, :, ::-1],
                 	metadata=sps_metadata,
                 	instance_mode=ColorMode.IMAGE_BW
        )
        out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
        frame = out.get_image()[:, :, ::-1]
        cv2.imshow('SPS video', frame)
        cv2.waitKey(1)
        publisher.publish(br.cv2_to_imgmsg(frame))
        rclpy.spin_once(node)
        
except KeyboardInterrupt:
    pass
input_video.release()

In [ ]:
rclpy.shutdown()